In [1]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 30.9 MB/s eta 0:00:00


In [2]:
from gurobipy import *

In [4]:
try:
  m = Model("CAU Logistics")

  cost_ij = [[8,14,32,29],[10,13,30,26]]
  cost_jk = [[17,18],[15,17]]
  capa_1 = [150,130]
  capa_2 = [100,120]
  demand = [140,140]

  x_var_names_1 = [["x"+str(i+1)+str(j+3) for j in range(len(cost_ij[0]))] for i in range(len(cost_ij))]
  x_var_names_2 = [["x"+str(i+3)+str(j+5) for j in range(len(cost_jk[0]))] for i in range(len(cost_jk))]

  x_var_1 = []
  for i in range(len(cost_ij)):
    x_var_1.append([])
    for j in range(len(cost_ij[0])):
      x_var_1[i].append(m.addVar(obj = cost_ij[i][j], vtype=GRB.INTEGER, name="(%s)" % (x_var_names_1[i][j])))

  x_var_2 = []
  for i in range(len(cost_jk)):
    x_var_2.append([])
    for j in range(len(cost_jk[0])):
      x_var_2[i].append(m.addVar(obj = cost_jk[i][j], vtype=GRB.INTEGER, name="(%s)" % (x_var_names_2[i][j])))
  
  m.modelSense = GRB.MINIMIZE
  m.update()

  for i in range(len(cost_ij)):
    m.addConstr(quicksum(x_var_1[i][j] for j in range(len(cost_ij[i]))) == capa_1[i], "capacity at manufacturer")

  for i in range(len(cost_jk)):
    m.addConstr(quicksum(x_var_2[i][j] for j in range(len(cost_jk[i]))) <= capa_2[i], "capacity at intermediate")

  for j in range(len(demand)):
    m.addConstr(quicksum(x_var_1[i][j+2] for i in range(len(cost_ij))) + quicksum(x_var_2[i][j] for i in range(len(cost_jk))) == demand[j], "demand satisfaction")

  for j in range(len(cost_jk)):
    m.addConstr(quicksum(x_var_1[i][j] for i in range(len(cost_ij))) == quicksum(x_var_2[j][k] for k in range(len(demand))), "flow balance")

  m.optimize()

  for v in m.getVars():
    print(v.varName, v.x)
            
  print("Obj:", m.objVal)

except GurobiError:
  print("Error reported")

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 8 rows, 12 columns and 28 nonzeros
Model fingerprint: 0xfb122fc2
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Found heuristic solution: objective 8060.0000000
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 28 nonzeros
Variable types: 0 continuous, 12 integer (0 binary)

Root relaxation: objective 7.330000e+03, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0   